## Smart Signatures with Atomic Swaps
#### 06.4 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-22

* Actually get money of of the Vending machine

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the 5 accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import pandas as pd

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

## The Vending Machine
* So far so good, there is lots of ALGOS in the vending machine

#### How do we get our money?

## Exercise
* Change the vending machine so that we actually get our money

#### Step 0: Get the status before the swap
* Also fund accounts if need be
* https://bank.testnet.algorand.network
* https://testnet.algoexplorer.io/dispenser

In [ ]:
# Get the holdings of MyAlgo and Bob separately
myAlgo_holding=asset_holdings_df(algod_client, MyAlgo['public'])
bob_holding=asset_holdings_df(algod_client, Bob['public'])
# Merge in one data.frame using pandas merge
pd.merge(myalgo_holding, bob_holding,  how="outer", on=["asset-id", "unit", "name", "decimals"], suffixes=['MyAlgo','Bob'])

In [ ]:
# Store the correct ID for the WSC coin
WSC_idx=71140107                         # <---------- Update!!

#### Step 1a: Write down the conditions as a PyTeal program
Small complication: there is no 

In [ ]:
vending_condition = And (
    Global.group_size() == Int(2),                     # Vending is an atomic swap, hence 2 transactions
    Gtxn[0].type_enum() == TxnType.Payment,            # First TX is a payment TX ...
    Gtxn[0].xfer_asset() == Int(0),                    # ... in ALGOs
    Gtxn[1].type_enum() == TxnType.AssetTransfer,      # Second TX is an ASA transfer ...
    Gtxn[1].xfer_asset() == Int(WSC_idx),               # ... in the WSC coin
    Gtxn[0].amount() == Gtxn[1].asset_amount(),        # Exchange rate in SMALL units
    Gtxn[0].rekey_to() == Global.zero_address(),
    Gtxn[0].close_remainder_to() == Global.zero_address(),
    Gtxn[1].rekey_to() == Global.zero_address(),
    Gtxn[1].close_remainder_to() == Global.zero_address(),
)

optin_condition = And(
        Global.group_size() == Int(1),                # Opt-in is single transaction
        Txn.type_enum() ==TxnType.AssetTransfer,      # Opt-in is ASA transfer
        Txn.asset_amount() == Int(0),                 # Payout impossible, opt-in is OK
        Txn.rekey_to() == Global.zero_address(),
        Txn.close_remainder_to() == Global.zero_address(),
    )


# prepare reandom condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = Txn.fee() <= Int(1000)
    
vending_pyteal = And(
    random_cond, 
    fee_cond, 
    If(
        Global.group_size() == Int(1),        # condition
        optin_condition,                      # then-expression
        vending_condition                     # else-expression
        )
    )

#### Step 1b: Pyteal -> Teal

In [ ]:
vending_teal = compileTeal(vending_pyteal, Mode.Signature, version=3)
print(vending_teal)

In [ ]:
#### Step 1c: Teal -> Bytecode for AVM

In [ ]:
Vending = algod_client.compile(vending_teal)
Vending

#### Step 2: Veding must opt-In
* Veding needs to be funded first (for TX fee and min holding)
* Veding then makes a 0 coin transaction to itself

##### Step 2a: Funding

In [ ]:
# Step 2a.1: prepare transaction
sp = algod_client.suggested_params()

# How much? Min holdings + min holdings for 1 ASA + TX fee for a few several swaps
amt = int(0.1*1e6) + int(0.1*1e6) + int(20* 0.001*1e6)
txn = transaction.PaymentTxn(sender=MyAlgo['public'], sp=sp, 
                             receiver=Vending['hash'], amt=amt)

# Step 2a.(2+3+4): sign and send and wait ...
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

##### Step 2b: Opt-In

In [ ]:
# Steo 2b.1: Prepare
sp = algod_client.suggested_params()
txn = AssetTransferTxn(Vending['hash'], sp, Vending['hash'], 0, WSC_idx)

# Steo 2b.2: Sign
encodedProg = Vending['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 2b.3 Send
txid = algod_client.send_transaction(stxn)

# Step 2b.4 Wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 3: MyAlgo puts 15 (full) WSC into the Vending machine
* This is a simple AssetTransferTxn

In [ ]:
### Step 3.1: prepare and create TX

# Deal with SMALL units
WSC_decimals = algod_client.asset_info(WSC_idx)['params']['decimals']
amt = int( 15 * 10**WSC_decimals )             # <--------- 15 WSC coins in SMALL units

sp = algod_client.suggested_params()
txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Vending['hash'],               
    amt=amt,
    index=WSC_idx)                        # <-----  We are sending WSC

# Step 3.2 and 3.3: sign and send
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)

# Step 3.4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

## The vending machine is now ready

#### Step 4: Opt-In for Bob
* If Bob is a first-time buyer, he has to opt in first
* In a real-world example, we would check the holdings of Bob and ask him to opt in only if he does not hold the token

In [ ]:
# Step 4.1: Prepare transaction
sp = algod_client.suggested_params()
txn = AssetTransferTxn(
    sender=Bob['public'],                 # <------- From Bob ...
    sp=sp,
    receiver=Bob['public'],               # <------- ... to Bob
    amt=int(0),
    index=WSC_idx)                        # <----- Correct asset_idx

# Step 4.2 and 4.3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob
txid = algod_client.send_transaction(stxn)

# Step 4.4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 5: Bob prepares the entire Atomic Swap
* Bob sends 5 micro-ALGOS
* He asks for 5 mini-WSC

In [ ]:
# Step 5.1a: Prepare ALGO payment TXN from Bob to Veding
sp = algod_client.suggested_params()
amt_1 = int(5)                     # microalgos!!! 
txn_1 = PaymentTxn(Bob['public'], sp, Vending['hash'],amt_1)

# Step 5.1b: Prepare WSC transfer from Vending to Bob
amt_2 = int(5)                      # mini-WSC coin
txn_2 = AssetTransferTxn(Vending['hash'], sp, Bob['public'], amt_2, WSC_idx)

In [ ]:
# Step 5.1c: create TX group
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid     # add group_id to each transactions
txn_2.group = gid
print( base64.b32encode(gid).decode() )      # This is the gid

In [ ]:
# Step 5.2a: Bob signs txn_1
stxn_1 = txn_1.sign(Bob['private'])

# Step 5.2b: Bob asks Smart signature to authorize txn_2
encodedProg = Vending['result'].encode()              
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

In [ ]:
# Step 5.3: assemble transaction group and send
signed_group =  [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

In [ ]:
# Step 5d: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

**Quick check** what is the `txid` here?
* Click on Group ID in Algoexplorer

In [ ]:
print('https://testnet.algoexplorer.io/tx/'+txid)

#### Get the status after the swap

In [ ]:
myAlgo_holding=asset_holdings_df(algod_client, MyAlgo['public'])
bob_holding=asset_holdings_df(algod_client, Bob['public'])
pd.merge(myalgo_holding, bob_holding,  how="outer", on=["asset-id", "unit", "name", "decimals"], suffixes=['MyAlgo','Bob'])